In [1]:
from transformers import PreTrainedModel, LlamaConfig, LlamaModel, LlamaTokenizer
import torch.nn as nn
import torch
from typing import Optional, List

class LlamaRewardModel(PreTrainedModel):
    config_class = LlamaConfig
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.regression_head = nn.Linear(self.config.hidden_size, 1, bias=False)

    def forward( # args are the same as LlamaForCausalLM
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        transformer_outputs = self.model(
                                input_ids,
                                attention_mask=attention_mask,
                                position_ids=position_ids,
                                past_key_values=past_key_values,
                                inputs_embeds=inputs_embeds,                               
                            )

        hidden_states = transformer_outputs[0]
        rewards = self.regression_head(hidden_states).squeeze(-1)
        
        return rewards

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-19 19:01:24,217] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer = LlamaTokenizer.from_pretrained("openbmb/UltraRM-13b")
model = LlamaRewardModel.from_pretrained("openbmb/UltraRM-13b", device_map=0, torch_dtype=torch.bfloat16)
model.eval()

tokenizer.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1.84M/1.84M [00:00<00:00, 30.9MB/s]
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:20<00:00,  6.88s/it]


LlamaRewardModel(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [9]:
ultrarm_template = """Human: {instruction}

Assistant: {completion}"""

dataset = [ # cases in webgpt
  {
      "chosen":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Sean Connery's character, fictional British Secret Service agent James Bond, in the movie Goldfinger, stated that he preferred his martini to be \"shaken, not stirred\". [1] Some believe that Bond ordered his martini shaken because of the vodka it contained, as vodka was, for the most part, refined from potatoes (cheaper brands)",
      "rejected":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Fleming's fictional British Secret Service agent James Bond orders his martini cocktail shaken, not stirred [1]. Bond's preferences for his martini are carried over to the films, where his orders are seen in both the 1961 film Dr. No and the 2006 film Casino Royale [1, 2]. In both films, Bond's subordinates copy his order, telling the bartender to keep the fruit with their drinks [2]. However, in the 2006 film, Bond appears irritated when the bartender asks if he would like his drink shaken or stirred [2]."},
  {"chosen":"Human: Sural relates to which part of the body?\nAssistant: The sural region is the muscular swelling of the back of the leg below the knee, formed chiefly by the bellies of the gastrocnemius and soleus muscles [1,2].",
   "rejected":"Human: Sural relates to which part of the body?\nAssistant: The Sural nerve runs down the side of the leg near the small saphenous vein, then passes forward below the lateral malleolus and continues on the outside of the foot as the lateral dorsal cutaneous nerve, which then communicates with the intermediate dorsal cutaneous nerve, which branches off to the side of the foot. [1]"}
]

In [10]:
chosens = []
rejs = []
for example in dataset:
    inputs = tokenizer(example["chosen"], return_tensors="pt").to(model.device)
    print(len(inputs.input_ids[0]))
    chosens.extend(model(**inputs).tolist())
    inputs = tokenizer(example["rejected"], return_tensors="pt").to(model.device)
    rejs.extend(model(**inputs).tolist())
    #print(chosen_reward - rejected_reward)

# Output 1: 2.4158712085336447
# Output 2: 0.1896953582763672

108
63


In [5]:
print(chosens[0])

[15.5, -3.703125, -4.75, -7.09375, -8.0625, -8.9375, -8.6875, -8.6875, -7.3125, -7.40625, -7.15625, -6.59375, -7.71875, -7.71875, -6.21875, -5.3125, -5.40625, -5.0625, -5.875, -6.09375, -7.0625, -8.125, -7.875, -6.375, -6.84375, -6.5, -4.0625, -6.09375, -5.34375, -4.1875, -4.96875, -6.8125, -5.53125, -5.0625, -5.3125, -5.375, -4.75, -4.125, -1.6640625, -1.8671875, -3.1875, -3.28125, -2.953125, -3.359375, -4.65625, -1.1875, -1.9296875, -2.3125, -2.578125, -3.515625, -4.09375, -3.828125, -5.5625, -3.328125, -4.125, -4.03125, -3.515625, -4.96875, -3.21875, -4.0, -4.28125, -3.640625, -1.421875, 0.1435546875, -0.65234375, -1.15625, -0.08203125, -1.9765625, -2.28125, -2.359375, -2.3125, -2.8125, -3.0, -3.25, -2.875, -3.875, -2.671875, -2.953125, -3.40625, -3.3125, -3.671875, -4.375, -3.125, -3.28125, -2.78125, -2.953125, -3.359375, -3.59375, -3.953125, -3.390625, -3.765625, -3.359375, -3.75, -3.46875, -1.34375, -3.21875, -3.328125, -3.4375, -2.65625, -3.171875, -3.5, -2.921875, -2.734375, -3

In [11]:
print(chosens[0])

[15.6875, -3.765625, -4.875, -7.21875, -8.0625, -8.875, -8.6875, -8.75, -7.375, -7.375, -7.1875, -6.5625, -7.8125, -7.6875, -6.3125, -5.40625, -5.40625, -5.125, -5.875, -6.1875, -7.15625, -8.1875, -7.90625, -6.34375, -6.84375, -6.5, -4.1875, -6.15625, -5.40625, -4.28125, -5.0, -6.9375, -5.40625, -5.15625, -5.28125, -5.46875, -4.78125, -4.1875, -1.734375, -2.09375, -3.015625, -3.3125, -3.015625, -3.359375, -4.8125, -1.2890625, -1.9921875, -2.421875, -2.578125, -3.4375, -4.0625, -3.875, -5.625, -3.390625, -4.09375, -4.09375, -3.546875, -5.03125, -3.25, -3.96875, -4.3125, -3.578125, -1.453125, 0.2353515625, -0.59375, -1.1640625, -0.06640625, -1.9453125, -2.46875, -2.34375, -2.28125, -2.875, -3.046875, -3.296875, -2.796875, -3.84375, -2.796875, -2.984375, -3.40625, -3.21875, -3.640625, -4.3125, -3.1875, -3.375, -2.78125, -2.953125, -3.359375, -3.671875, -3.953125, -3.4375, -3.90625, -3.359375, -3.671875, -3.703125, -1.3984375, -3.140625, -3.4375, -3.421875, -2.65625, -3.09375, -3.453125, -